In [9]:
import cv2 
import numpy as np 
import math
import os
from PIL import Image, ImageDraw

In [25]:
# Defining Parameters

totalImageCount = 1 #the number of gifs we want to make
imageIndices = [str(i) for i in range(1,totalImageCount+1)] # the file names are renamed to format "1_front.jpg"
frames = 120 # Total frames we want to use, larger if smoother is required, but file size will be larger
buffer_px = 100 # the size of the black frame in pixels
zoom_factor = 50

In [26]:
# Definining the directory paths
# For some reason imread and imwrite works more consistently when we os.chdir() to the directories

current_directory = os.getcwd()
input_path = current_directory + '/inputs'
output_path = current_directory+'/outputs'
gif_output_path = current_directory+'/gif_outputs'


# Main loop, for each pair of images (front and back)
for imageIndex in imageIndices:
    
    # for each frame, we want to generate an image
    for i in range(0, frames):
        
        # every frame we rotate the image a bit depending on how many frames we have
        angleIncrement = 2*math.pi/frames
        angle = angleIncrement * i

        # between 90 degrees and 270 degrees, we will use the BACK face, else use the front face
        face = "back" if (angle>= math.pi/2 and angle <= 3*math.pi/2) else "front"
        
        # now we go into the inputs folder and get the face required
        os.chdir(current_directory)
        img = cv2.imread(input_path + "/" + imageIndex + "_" + face + ".jpg")
        
        # if the image is the backside, we have to make sure it is flipped horizontally due to transformation
        if face == "back":
            img = cv2.flip(img, 1)
        
        # resize the image so our outputs are consistent in size, we scaled width to at most be 600px
        length, width, _ = img.shape
        down_points = (int(600*width/length),600) 
        img = cv2.resize(img, down_points)

        # add a buffer black frame of buffer_px size, so when it rotates, our image remains in frame
        img = cv2.copyMakeBorder(img, buffer_px, buffer_px, buffer_px, buffer_px, cv2.BORDER_CONSTANT, value=[0,0,0,0])

        # pinpoint the coordinates of the original corners, this is used to trace the trajectory depending on angle
        bottomLeft = [buffer_px, buffer_px+length]
        bottomRight = [buffer_px+width, buffer_px+length]
        topRight = [buffer_px+width, buffer_px]
        topLeft = [buffer_px, buffer_px]

        input_pts = np.float32([bottomLeft, bottomRight, topRight, topLeft])

        # now trace where the four corners will be at depending on the angle
        transbottomLeft = [buffer_px + width*math.pow(math.sin(angle/2),2) , bottomLeft[1] + zoom_factor*math.sin(angle)]
        transbottomRight = [bottomRight[0] - width*math.pow(math.sin(angle/2),2), bottomRight[1] - zoom_factor*math.sin(angle) ]
        transtopRight = [topRight[0] - width*math.pow(math.sin(angle/2),2) , buffer_px + zoom_factor*math.sin(angle)]
        transtopLeft = [buffer_px + width*math.pow(math.sin(angle/2),2), buffer_px - zoom_factor*math.sin(angle)]
        
        output_pts = np.float32([transbottomLeft, transbottomRight, transtopRight, transtopLeft])
        
        # use perspective transform to fit our image into the transformed points
        M = cv2.getPerspectiveTransform(input_pts, output_pts)
        out = cv2.warpPerspective(img,M,(width+2*buffer_px, length+2*buffer_px),flags=cv2.INTER_LINEAR)

        # go to our output folder and save the frame
        os.chdir(output_path)
        fileName = imageIndex + '_'+str(i)+'.png'
        cv2.imwrite(fileName, out, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
        
    # now that we have all frames in the output folder, we put them in an array
    frameImages = []
    
    for f in range(frames): 
        tempImg = Image.open(output_path+'//'+nft+'_'+str(f)+'.png')
        frameImages.append(tempImg)
        
    # stitch them together into a GIF and save it to the gif_outputs folder
    im1 = frameImages[0]
    os.chdir(gif_output_path)
    im1.info['duration'] = 60
    im1.save(imageIndex+".gif", save_all=True, append_images=frameImages[1:], loop=0, optimize=True,quality=80)
    
    # return back to the original directory
    os.chdir(current_directory)
    
    # delete all the frame images to save disk storage
    for f in range(frames): 
        if os.path.exists(output_path+'//'+imageIndex+'_'+str(f)+'.png'):
            os.remove(output_path+'//'+imageIndex+'_'+str(f)+'.png')